# My Iggy

A simple `iggy`-like system.

In [85]:
%%clingo -n 0

color_level(11).
color(-L..L) :- color_level(L).
    
sum(S) :- S = #sum{C : color(C)}.
count(T) :- T = #count{C : color(C)}.

divide(Q) :- count(T), Q = T/3.
divide0(Q) :- count(T), Z = #sum{C : color(C), C > 100}, Q = Z/T.

strange_mod(-1*(-1/3)).

clingo version 5.4.0
Reading from stdin
Solving...
Answer: 1
color_level(11) color(-11) color(-10) color(-9) color(-8) color(-7) color(-6) color(-5) color(-4) color(-3) color(-2) color(-1) color(0) color(1) color(2) color(3) color(4) color(5) color(6) color(7) color(8) color(9) color(10) color(11) sum(0) count(23) divide(7) divide0(0) strange_mod(0)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.001s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.001s


In [113]:
%%clingo -n 0 --project --parallel-mode 8

% Description of the signed directed graph.
node(arcA; fnr; rpsP; rpmC).

% Possible signs of an edge.
sign(-1; 1).

% Given edges and their sign.
edge(arcA, fnr, -1).
edge(arcA, rpmC, -1).
edge(fnr, rpmC, 1).
edge(fnr, rpsP, -1).

% Deduce the input nodes.
input(J) :- node(J), not influence(J, _).

% ==CHANGE: There are no real "colors" for nodes anymore.==
% We use signed weights instead.
color_level(3).
color(-L..L) :- color_level(L).

% Measurements of a node in 5 levels: -, dn, 0, up, +.
% Taking values (-2, -1, 0, 1, 2) to
% simplify description of RULE 1 below.
level(-2; -1; 0; 1; 2).

% Observed nodes and their color.
% observed(rpsP, 1).
observed(rpsP, -2).
observed(rpmC, -1).

% Each node can only have 1 color.
1 {node_color(N, C): color(C)} 1 :- node(N).
% forbidden(N, C) :- node_color(N, D), color(C), color(D), D != C.

% Propagate effects forward.
% node_color(J, C) :- node(J), color(C), not forbidden(J, C).

% ==CHANGE: RULE 1: Observation satisfaction==
% For each observed node, the model behavior must match
% the biological observation data.
% That is, given a tolerance threshold C0 > 0:

color_tol(L/2) :- color_level(L).

% ~~Where the model says should be 1, the data should >= 1;~~
% ==Where the model says should >= C0, the data should >= 1 ;==
forbidden(N, C) :- observed(N, L), L < 1,
    color_tol(C0), color(C), C >= C0.

% Where the model says should be -C0 <= model <= C0,
% the data should be -1 <= data <= 1;
forbidden(N, C) :- observed(N, L), |L| > 1,
    color_tol(C0), color(C), |C| <= C0.

% ~~Where the model says should be -1, the data should <= -1.~~
% ==Where the model says should be <= -C0, the data should <= 1.==
forbidden(N, C) :- observed(N, L), L > -1,
    color_tol(C0), color(C), C <= -C0.

% ==CHANGE: RULE 2: Justification of the change in one predecessor==
% For any non-input node, its change must be explained by the influence
% ~~of at least one of its predecessors in the graph.~~
% ==of all of its predecessors in the graph.==
% More precisely, null changes can always occur independently of the
% state of the predecessor nodes. However, for observed changes in 
% node activation levels, each must be explainable by an influence
% (with the appropriate sign) of at least one predecessor.

% For any non-input node J with color C,
% ~~there must be at least one instance of influence(J, C).~~
% ==its new color is the weighted mean of its precessors'==
% ==influences when surpassing a limit Q0 > 0, and 0 otherwise.==
color_activ(L/2) :- color_level(L).

node_color_raw(J, S, T) :- node(J), not input(J), influence(J),
    S = #sum{F : influence(J, F)},
    T = #sum{|F| : influence(J, F)}.

node_color(J, S/T) :- T != 0, node_color_raw(J, S, T),
    not node_color(J, 0).
node_color(J, 0) :- T != 0, node_color_raw(J, S, T),
    color_activ(Q0), |S/T| < Q0.

% We define the influence of node i on node j as:
% influence(i, j) := sign(i) * sign(edge(i, j))
influence(I, J, P*Q) :- edge(I, J, P), node_color(I, Q).
influence(J, C) :- influence(I, J, C).
influence(J) :- influence(J, C).

% RULE 3: Justification of zero change
% A node can only exhibit zero change if it receives no or conflicting
% influences. This constraint therefore assumes that each influence has
% an effect, and that only contradictory influences can cancel each
% other out.
forbidden(J, 0) :- node(J), influence(J),
    #sum{F : influence(J, F)} != 0.

% Repair model
% Each new influence must be consistent with itself,
% and at the same time inconsistent with the other.
new_color(C) :- color(C), color_activ(Q0), C >= |Q0|.
new_influence(J, C) :- not not new_influence(J, C),
    node(J), new_color(C).
1 {new_influence(J, C): new_color(C)} :- node(J).
    
influence(J, C) :- new_influence(J, C).

% Minimize the number of new influences (fixes) added to the graph.
#minimize{ 1, (J, C) : new_influence(J, C) }.

#show node_color_raw/3.
#show node_color/2.
#show influence/2.
% #show new_influence/2.

clingo version 5.4.0
Reading from stdin
Solving...
Progression : [1;inf]
Progression : [2;inf]
Progression : [3;inf]
Progression : [4;inf]
Answer: 1
influence(arcA,1) influence(arcA,2) influence(arcA,3) influence(fnr,1) influence(rpsP,3) influence(rpmC,1) node_color(fnr,1) node_color(arcA,0) influence(rpmC,0) influence(fnr,0) influence(rpsP,-1) node_color_raw(arcA,6,6) node_color_raw(fnr,1,1) node_color_raw(rpsP,2,4) node_color_raw(rpmC,1,1) node_color(rpsP,0) node_color(rpmC,1)
Optimization: 6
Answer: 2
influence(arcA,1) influence(fnr,1) influence(rpsP,3) influence(rpmC,1) node_color(fnr,1) node_color(arcA,0) influence(rpmC,0) influence(fnr,0) influence(rpsP,-1) node_color_raw(arcA,1,1) node_color_raw(fnr,1,1) node_color_raw(rpsP,2,4) node_color_raw(rpmC,1,1) node_color(rpmC,0) node_color(rpsP,0)
Optimization: 4
OPTIMUM FOUND

Models       : 2
  Optimum    : yes
Optimization : 4
Calls        : 1
Time         : 25.280s (Solving: 23.89s 1st Model: 23.81s Unsat: 0.02s)
CPU Time     : 191

*** Warn : (clingo): Projection: Optimization may depend on enumeration order.
